In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten, Dense

2024-10-15 10:44:30.040851: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-15 10:44:30.147928: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 10:44:30.147981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 10:44:30.166497: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 10:44:30.214607: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#Loading dataset

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [3]:
#Preprocessing

train_images = train_images.reshape((60000, 784))
train_images = train_images.astype('float32')/255
test_images = test_images.reshape((10000, 784))
test_images = test_images.astype('float32')/255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [4]:
from keras import models
from keras.layers import Dense

In [13]:
from tensorflow.keras.optimizers import Adam, RMSprop, Nadam

In [15]:
class EBV_Layer(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(EBV_Layer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.kernal = self.add_weight(name='kernal',
                                      shape = (input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(EBV_Layer, self).build(input_shape)
    def call(self, x):
        kernal_norm = tf.norm(self.kernal, axis=0, keepdims=True)
        kernal_norm = tf.maximum(kernal_norm, tf.keras.backend.epsilon())
        kernal_unit_norm = self.kernal/kernal_norm
        output = tf.matmul(x, kernal_unit_norm)
        return output
    
model1 = models.Sequential()
# model1.add(Flatten())
model1.add(Dense(512, input_dim=784, activation='relu'))
model1.add(Dense(256, activation="relu"))
model1.add(Dense(128, activation="relu"))
model1.add(Dense(64, activation="relu"))
model1.add(EBV_Layer(10, input_shape=(64,)))

model1.compile(optimizer=Nadam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['acc'])


model1.summary()


history = model1.fit(train_images,
                     train_labels,
                     batch_size=32,
                     epochs=6,
                     validation_split=0.1
                     )

result = model1.evaluate(test_images, test_labels)
print("Accuracy: ", result[1], " loss: ", result[0])

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 512)               401920    
                                                                 
 dense_33 (Dense)            (None, 256)               131328    
                                                                 
 dense_34 (Dense)            (None, 128)               32896     
                                                                 
 dense_35 (Dense)            (None, 64)                8256      
                                                                 
 ebv__layer_8 (EBV_Layer)    (None, 10)                640       
                                                                 
Total params: 575040 (2.19 MB)
Trainable params: 575040 (2.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/6
1688/168